In [7]:
%pip install tensorflow keras-tuner


In [8]:

import tensorflow as tf
from tensorflow.keras import layers, models, datasets
from tensorflow.keras.utils import to_categorical
import keras_tuner as kt



In [9]:

def load_and_preprocess_data():
    (x_train, y_train), (x_test, y_test) = datasets.mnist.load_data()
    x_train, x_test = x_train / 255.0, x_test / 255.0
    x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
    x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)
    y_train = to_categorical(y_train, 10)
    y_test = to_categorical(y_test, 10)
    return (x_train, y_train), (x_test, y_test)



In [10]:

def residual_block(x, filters, kernel_size=3, stride=1):
    shortcut = x
    x = layers.Conv2D(filters, kernel_size=kernel_size, strides=stride, padding='same', activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Conv2D(filters, kernel_size=kernel_size, strides=stride, padding='same')(x)
    x = layers.BatchNormalization()(x)
    if stride != 1:
        shortcut = layers.Conv2D(filters, kernel_size=1, strides=stride, padding='same')(shortcut)
        shortcut = layers.BatchNormalization()(shortcut)
    x = layers.add([x, shortcut])
    x = layers.Activation('relu')(x)
    return x


In [11]:

def model_builder(hp):
    inputs = layers.Input(shape=(28, 28, 1))
    x = layers.Conv2D(
        filters=hp.Int('filters', min_value=32, max_value=128, step=32),
        kernel_size=hp.Choice('kernel_size', values=[3, 5]),
        padding='same', activation='relu')(inputs)
    x = layers.BatchNormalization()(x)
    x = residual_block(x, filters=hp.Int('filters', min_value=32, max_value=128, step=32))
    x = residual_block(x, filters=hp.Int('filters', min_value=32, max_value=128, step=32))
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(10, activation='softmax')(x)
    model = models.Model(inputs, x)
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=hp_learning_rate),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model



In [12]:

(x_train, y_train), (x_test, y_test) = load_and_preprocess_data()
tuner = kt.Hyperband(model_builder,
                     objective='val_accuracy',
                     max_epochs=10,
                     factor=3,
                     directory='/content/drive/MyDrive/Resnet Trails',
                     project_name='resnet_mnist_tuning')
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
tuner.search(x_train, y_train, epochs=10, validation_data=(x_test, y_test), callbacks=[stop_early])

# Retrieve the best hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

# Print the best hyperparameters
print(f"""
The optimal number of filters is {best_hps.get('filters')},
the optimal kernel size is {best_hps.get('kernel_size')},
and the optimal learning rate is {best_hps.get('learning_rate')}.
""")



Trial 30 Complete [00h 07m 25s]
val_accuracy: 0.9902999997138977

Best val_accuracy So Far: 0.9905999898910522
Total elapsed time: 01h 06m 08s

The optimal number of filters is 96,
the optimal kernel size is 5,
and the optimal learning rate is 0.01.



In [13]:

model = tuner.hypermodel.build(best_hps)
history = model.fit(x_train, y_train, epochs=10, validation_data=(x_test, y_test))
loss, accuracy = model.evaluate(x_test, y_test, verbose=2)
print("Final test accuracy: {accuracy:.4f}")
model.save('best_resnet_mnist_model.h5')



Epoch 1/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 39s 16ms/step - accuracy: 0.8773 - loss: 0.4017 - val_accuracy: 0.9203 - val_loss: 0.2773
Epoch 2/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 35s 15ms/step - accuracy: 0.9768 - loss: 0.0732 - val_accuracy: 0.9617 - val_loss: 0.1280
Epoch 3/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 28s 15ms/step - accuracy: 0.9822 - loss: 0.0575 - val_accuracy: 0.9718 - val_loss: 0.0942
Epoch 4/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 28s 15ms/step - accuracy: 0.9833 - loss: 0.0514 - val_accuracy: 0.9741 - val_loss: 0.0916
Epoch 5/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 28s 15ms/step - accuracy: 0.9871 - loss: 0.0429 - val_accuracy: 0.9805 - val_loss: 0.0666
Epoch 6/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 28s 15ms/step - accuracy: 0.9883 - loss: 0.0375 - val_accuracy: 0.9822 - val_loss: 0.0615
Epoch 7/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 28s 15ms/step - accuracy: 0.9906 - loss: 0.0326 - val_accuracy: 0.9721 - val_loss: 0.0896
Epoch 8/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 28s 15ms/step - accuracy: 0.9911 -

Final test accuracy: {accuracy:.4f}


In [14]:
print(f"Final test accuracy: {accuracy:.4f}")

Final test accuracy: 0.9873
